In [1]:
import os
import vertexai
import pandas as pd
from ast import literal_eval
import chromadb
from dotenv import load_dotenv
vertexai.init(project="smart-agent-414407")
from vertexai.preview.generative_models import GenerativeModel
from vertexai.language_models import TextEmbeddingModel
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)
load_dotenv()

True

In [2]:
# import libs and env
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
import dotenv
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display, Markdown
from termcolor import colored  
import wget
import os
import chromadb
from ast import literal_eval
#GPT_MODEL = "gpt-4-1106-preview"
GPT_MODEL = "gpt-3.5-turbo-0125"
EMBEDDING_MODEL = "text-embedding-3-small"
load_dotenv()  

True

In [3]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

In [4]:
model = GenerativeModel("gemini-1.0-pro")

In [5]:
# Placeholder function for generating embeddings using OpenAI
# def generate_embedding(text) -> list:
#     """
#     Generates an embedding vector for the given text using OpenAI's embedding API.
#     Replace this with an actual call to openai.Embedding.create() in your code.
#     """
#     """Text embedding with a Large Language Model."""
#     model = TextEmbeddingModel.from_pretrained("textembedding-gecko@003")
#     embeddings = model.get_embeddings([text])
#     for embedding in embeddings:
#         vector = embedding.values
    
#     return vector
from openai import OpenAI
client = OpenAI()

# Placeholder function for generating embeddings using OpenAI
def generate_embedding(text):
    """
    Generates an embedding vector for the given text using OpenAI's embedding API.
    Replace this with an actual call to openai.Embedding.create() in your code.
    """
    response = client.embeddings.create(
        input=text,
        model=EMBEDDING_MODEL
    )

    return response.data[0].embedding  # Assuming embedding size is 768

def process_md_files_to_df(folder_path):
    """
    Processes all markdown files in the given folder and compiles them into a DataFrame.
    
    :param folder_path: Path to the folder containing MD files.
    :return: A DataFrame with the columns id, file_path, title, text, title_vector, content_vector, vector_id.
    """
    data = []
    
    for idx, filename in enumerate(os.listdir(folder_path)):
        if filename.endswith(".md"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                title = os.path.splitext(filename)[0]
                title_vector = generate_embedding(title)
                content_vector = generate_embedding(content)
                
                # Append to data list
                data.append({
                    "id": idx + 1,
                    "file_path": file_path,
                    "title": title,
                    "text": content,
                    "title_vector": title_vector,
                    "content_vector": content_vector,
                    "vector_id": idx  # Assuming vector_id starts from 0
                })
    
    # Convert list to DataFrame
    df = pd.DataFrame(data)
    return df

# Example usage
folder_path = "docs"

In [6]:
article_df = process_md_files_to_df(folder_path)

In [7]:
article_df.to_csv('docs/oa-docs.csv', index=False)

In [8]:
article_df = pd.read_csv('docs/oa-docs.csv')
article_df.head()

,id,file_path,title,text,title_vector,content_vector,vector_id
0,1,docs/serbia.md,serbia,# Serbia\n\n![Serbia Landscape](URL_to_an_imag...,"[-0.012984967790544033, -0.03350071609020233, ...","[-0.028921548277139664, -0.01833852380514145, ...",0
1,3,docs/corporate-actions.md,corporate-actions,# Standard Operating Procedure (SOP) for Corpo...,"[0.04675452411174774, 0.02118518389761448, 0.0...","[0.006607677787542343, 0.06554270535707474, 0....",2
2,4,docs/trade-post-settlement.md,trade-post-settlement,# Standard Operating Procedure (SOP) for Trade...,"[0.010448666289448738, -0.025498153641819954, ...","[-0.004678166471421719, 0.03849324211478233, 0...",3
3,5,docs/polanski.md,polanski,# Polanski: Serbia's Rising Tennis Star\n\n![P...,"[-0.0472092479467392, 0.06010713428258896, -0....","[-0.04051847383379936, 0.030533114448189735, -...",4
4,6,docs/trade-pre-matching.md,trade-pre-matching,# Standard Operating Procedure for Trade Pre-M...,"[-0.014318353496491909, 0.014613003470003605, ...","[-0.030199365690350533, 0.03870762139558792, 0...",5


In [9]:
# Read vectors from strings back into a list
article_df['title_vector'] = article_df.title_vector.apply(literal_eval)
article_df['content_vector'] = article_df.content_vector.apply(literal_eval)

# Set vector_id to be a string
article_df['vector_id'] = article_df['vector_id'].apply(str)

In [10]:
chroma_client = chromadb.EphemeralClient() # Equivalent to chromadb.Client(), ephemeral.
# Uncomment for persistent client
# chroma_client = chromadb.PersistentClient()

In [11]:
import chromadb.utils.embedding_functions as embedding_functions
#vertex_key = os.getenv("VERTEX_API_KEY")
#embedding_function = embedding_functions.GoogleVertexEmbeddingFunction(api_key=vertex_key)

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"), model_name=EMBEDDING_MODEL)

sop_content_collection = chroma_client.create_collection(name='content-v2', embedding_function=embedding_function)
sop_title_collection = chroma_client.create_collection(name='title-v2', embedding_function=embedding_function)

# Add the content vectors
sop_content_collection.add(
    ids=article_df.vector_id.tolist(),
    embeddings=article_df.content_vector.tolist(),
)

# Add the title vectors
sop_title_collection.add(
    ids=article_df.vector_id.tolist(),
    embeddings=article_df.title_vector.tolist(),
)

In [12]:
def search_df(collection, query, max_results, dataframe):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances']) 
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'score':results['distances'][0],
                'title': dataframe[dataframe.vector_id.isin(results['ids'][0])]['title'],
                'content': dataframe[dataframe.vector_id.isin(results['ids'][0])]['text'],
                })
    
    return df

In [13]:
# Function to create the desired string from a DataFrame
def search_knowledge(query, articles, n, clearing_house="na", market="na"):
    df = search_df(
    collection=sop_title_collection,
    query=query,
    max_results=n,
    dataframe=articles
    )
    # Initialize an empty string
    result_string = ""
    
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Append the title and content to the result string, followed by a newline character
        result_string += f"{row['title']}\n{row['content']}\n"
    
    return result_string.strip()  # Use strip() to remove the last newline character

In [14]:
get_current_weather_func = FunctionDeclaration(
    name="get_current_weather",
    description="Get the current weather in a given location",
    parameters={
        "type": "object",
        "properties": {"location": {"type": "string", "description": "Location"}},
    },
)

In [15]:
search_knowledge_func = FunctionDeclaration(
    name="search_knowledge",
    description="to answer any question related to euroclear, tennis players, and countries",
    parameters={
    "type": "object",
    "properties": {
        "query": {
            "type": "string",
            "description": "your search query to retrieve knowledge from a vector database"
        },
    },
         "required": [
            "query"
      ]
  },
)

In [16]:
knowledge_tool = Tool(
    function_declarations=[search_knowledge_func],
)

In [17]:
weather_tool = Tool(
    function_declarations=[get_current_weather_func],
)

You can then instruct the model to generate content, include the `tool` that you just created, to generate a response:

In [18]:
GENERAL_ASSISTANT = """
You are Sage, a friendly and helpful agent who helps users using the given functions.
Here are the rules you have to strictly adhere to:
1. Don't make assumptions about what values to plug into functions. 
2. You will use the search tool to find relavent knowlege articles to create the answer.
3. Being smart in your research. If the search does not come back with the answer, rephrase the question and try again.
4. Review the result of the search and use it to guide your next search if needed.
5. If the question is complex, break down to smaller search steps and find the answer in multiple steps.
6. Answer ONLY with the facts from the search tool. If there isn't enough information, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
"""

In [19]:
model = GenerativeModel(
    "gemini-1.0-pro", generation_config={"temperature": 0}, tools=[knowledge_tool]
)
chat = model.start_chat(response_validation=False)

In [20]:
prompt = f"{GENERAL_ASSISTANT}"

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

IndexError: list index out of range

In [21]:
prompt = "what's the tourism website of the place where tennis player Polanski was born?"

response = chat.send_message(prompt)
response.candidates[0].content.parts[0]

function_call {
  name: "search_knowledge"
  args {
    fields {
      key: "query"
      value {
        string_value: "Where was tennis player Polanski born?"
      }
    }
  }
}

In [22]:
arguments = response.candidates[0].content.parts[0].function_call.args

In [23]:
arguments['query']

'Where was tennis player Polanski born?'

In [24]:
search_result = search_knowledge(arguments['query'], article_df, n=3)

In [134]:
api_response = {"source": search_result}

In [135]:
api_response

{'source': 'external-settlement\n## External settlement\n19/09/2022\n\n### What is an external settlement transaction?\n\nA transaction between:  \n* a Euroclear client\n* a counterparty holding an account in a domestic market settlement system (directly, or via their domestic agent)\n\nAn external transaction settles:  \n* via a link we set up with the domestic market\n* in the domestic market CSD when we process the settlement confirmation from the local market\n\n### Four types of links with domestic markets\n\n* **direct link** – we have a direct account in the domestic market CSD through which transactions with local counterparties may settle\n\n* **indirect link** - we have an account with a local sub-custodian that has an account with the CSD through which transactions with local counterparties may settle.\n\n* **direct link managed by a domestic market agent** - we have an account directly with the domestic market CSD, but this account is operated on our behalf by a domestic ma

In [136]:
response = chat.send_message(
    Part.from_function_response(
        name="search_knowledge",
        response={
            "content": api_response,
        },
    ),
)
response.candidates[0].content.parts[0]

text: " The types of payments for bridge settlement are:\n\n* **Against payment:** The settlement of a transaction is conditional upon the receipt of payment.\n* **Free of payment:** The settlement of a transaction is not conditional upon the receipt of payment."

In [138]:
Markdown(response.candidates[0].content.parts[0].text)

 The types of payments for bridge settlement are:

* **Against payment:** The settlement of a transaction is conditional upon the receipt of payment.
* **Free of payment:** The settlement of a transaction is not conditional upon the receipt of payment.